<a href="https://colab.research.google.com/github/sanskriti49/Sentimental-Analysis/blob/main/Sentiment_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importing libraries

In [2]:
!git clone https://github.com/sanskriti49/Sentimental-Analysis
import numpy as np
import pandas as pd

Cloning into 'Sentimental-Analysis'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 79 (delta 31), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 122.32 KiB | 941.00 KiB/s, done.
Resolving deltas: 100% (31/31), done.


###Importing dataset

In [3]:
url="https://raw.githubusercontent.com/sanskriti49/Sentimental-Analysis/refs/heads/main/data/a2_RestaurantReviews_FreshDump.tsv"
dataset = pd.read_csv(url, delimiter = '\t', quoting = 3)
dataset.head()

,Review
0,Spend your money elsewhere.
1,Their regular toasted bread was equally satisf...
2,The Buffet at Bellagio was far from what I ant...
3,"And the drinks are WEAK, people!"
4,-My order was not correct.


###Data cleaning

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus=[]

for i in range(0, len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [ ]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
cv = pickle.load(open(cvFile, "rb"))


In [ ]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

###Predictions (via sentiment classifier)

In [ ]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [ ]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

#Mapping predicted labels to sentiment labels
sentiments=["Negative" if label==0 else "Positive" for label in y_pred]

[0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0
 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [ ]:
# dataset['predicted_label'] = y_pred.tolist()
dataset['predicted_label'] =sentiments
dataset.head()

,Review,predicted_label
0,Spend your money elsewhere.,Negative
1,Their regular toasted bread was equally satisf...,Positive
2,The Buffet at Bellagio was far from what I ant...,Positive
3,"And the drinks are WEAK, people!",Negative
4,-My order was not correct.,Negative


In [ ]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)